In [3]:
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup
from fuzzywuzzy import process, fuzz

The School Based Health Alliance hosts a pdf file containing the State, Congressman's name and Twitter handle. Download and parse this into a Data Frame.

In [4]:
result = requests.get('https://www.sbh4all.org/wp-content/uploads/2019/04/116th-Congress-Twitter-Handles.pdf')
with open('116th-Congress-Twitter-Handles.pdf', 'wb') as f:
    f.write(result.content)

Using a PDF viewer open, copy with ctrl-a + ctrl-c, and paste the content into a .txt file. Save it as 116th-Congress-Twitter-Handles.txt.

In [5]:
with open('116th-Congress-Twitter-Handles.txt', 'r') as f:
    d = []
    # for each line in the text file, split the line into a list of words. then clean them
    for r in [fi.strip().split(' ') for fi in f.readlines()]:
        # take the first and last name. if the length of the word is 2 then its a
        # middle initial or some name modifier lie j. or w.
        fullname = [ri for ri in r[1:-1] if len(ri) > 2]
        fullname = ' '.join([fullname[0], fullname[-1]]) if len(fullname) > 0 else ''
        
        d.append({
            'Position':r[0],
            'Name':fullname,
            'Handle':r[-1]
        })
        
handles = pd.DataFrame(d).iloc[1:]

print(len(handles))
handles.head()

400


,Position,Name,Handle
1,MT,Steve Gillibrand,@gillibrandny
2,NY,Charles Kaine,N/A
3,VA,Mark Murphy,@SenMurphyOffice
4,DE,Thomas Rubio,@SenRubioPress
5,FL,Rick Wicker,@SenatorWicker


Note we only have 400 handles when we should have 537. Going with it.

Party affiliations are taken from [Wikipedia](https://en.wikipedia.org/wiki/116th_United_States_Congress).

In [6]:
r = requests.get('https://en.wikipedia.org/wiki/116th_United_States_Congress')
soup = BeautifulSoup(r.content)

assert soup is not None

In [7]:
state_senators_html = [
    a for a in soup.find_all('a') 
    if 'title' in a.attrs.keys() and 'List of United States senators' in a.attrs['title']
]
all_senators = []
for state in state_senators_html:
    state_name = state.text
    if state_name in ['Expelled or censured', 'Born outside the U.S.', 'Switched parties']:
        continue
        
    senator_list = state.parent.parent
    senator_list = senator_list.find_next_siblings()
    
    if senator_list is None or len(senator_list) == 0:
        continue
        
    for li in senator_list[0].find_all('li'):
        try:
            name = li.find_all('a')[0].attrs['title']
            if '(' in name:
                name = name[:name.find('(')].trim()
                
            affiliation = li.text[::-1][:li.text[::-1].find('(')][::-1][:-1]

            if len(affiliation) > 1:
                affiliation = affiliation[0]

            all_senators.append({
                'State': state_name,
                'Name': name,
                'Affiliation': affiliation
            })
        except:
            pass
    
all_senators = pd.DataFrame(all_senators)
all_senators['Body'] = 'Senate'
all_senators.head()

,State,Name,Affiliation,Body
0,Alabama,Richard Shelby,R,Senate
1,Alaska,Lisa Murkowski,R,Senate
2,Arizona,Kyrsten Sinema,D,Senate
3,Arizona,Martha McSally,D,Senate
4,Arizona,Mark Kelly,D,Senate


In [8]:
state_reps_html = [
    a for a in soup.find_all('a') 
    if 'title' in a.attrs.keys() and 'List of United States Representatives from' in a.attrs['title']
]
all_reps = []
for state in state_reps_html:
    state_name = state.text
#     if state_name in ['Expelled or censured', 'Born outside the U.S.', 'Switched parties']:
#         continue
        
    rep_list = state.parent.parent
    rep_list = rep_list.find_next_siblings()
    
    if rep_list is None or len(rep_list) == 0:
        continue
        
    for li in rep_list[0].find_all('li'):
        try:
            name = li.find_all('a')[1].attrs['title']
            if '(' in name:
                name = name[:name.find('(')].trim()
                
            aidx = li.text.find('(')
            if aidx is not None and aidx >= 0:
                affiliation = li.text[aidx+1:aidx+2]
            else:
                affiliation = None

            all_reps.append({
                'State': state_name,
                'Name': name,
                'Affiliation': affiliation
            })
            affiliation = ''
        except:
            pass
    
all_reps = pd.DataFrame(all_reps)
all_reps['Body'] = 'House of Representatives'
all_reps.head()

,State,Name,Affiliation,Body
0,Alabama,Bradley Byrne,R,House of Representatives
1,Alabama,Martha Roby,R,House of Representatives
2,Alabama,Robert Aderholt,R,House of Representatives
3,Alabama,Mo Brooks,R,House of Representatives
4,Alabama,Terri Sewell,D,House of Representatives


In [9]:
legislature = pd.concat([all_senators, all_reps], axis=0)
legislature.head()

,State,Name,Affiliation,Body
0,Alabama,Richard Shelby,R,Senate
1,Alaska,Lisa Murkowski,R,Senate
2,Arizona,Kyrsten Sinema,D,Senate
3,Arizona,Martha McSally,D,Senate
4,Arizona,Mark Kelly,D,Senate


Cross-Join and Fuzzy Match on the Name

In [10]:
joined = []
for handle in handles.itertuples():
    for person in legislature.itertuples():
        match_score = fuzz.ratio(handle.Name, person.Name)

        joined.append({
            'Position': handle.Position,
            'StateName': person.State,
            'Name_x': handle.Name,
            'Name_y': person.Name,
            'Handle': handle.Handle,
            'Affiliation': person.Affiliation,
            'Body': person.Body,
            'Score': match_score
        })

joined = pd.DataFrame(joined)
joined.head()

,Position,StateName,Name_x,Name_y,Handle,Affiliation,Body,Score
0,MT,Alabama,Steve Gillibrand,Richard Shelby,@gillibrandny,R,Senate,27
1,MT,Alaska,Steve Gillibrand,Lisa Murkowski,@gillibrandny,R,Senate,13
2,MT,Arizona,Steve Gillibrand,Kyrsten Sinema,@gillibrandny,D,Senate,27
3,MT,Arizona,Steve Gillibrand,Martha McSally,@gillibrandny,D,Senate,20
4,MT,Arizona,Steve Gillibrand,Mark Kelly,@gillibrandny,D,Senate,23


Sort by Name_y, Score descending and take the highest match for each.

In [11]:
# take the top three matches for either side of the join
joined['RankY'] = joined.sort_values(by=['Name_y', 'Score'], ascending=False).groupby('Name_y').cumcount()
joined['RankX'] = joined.sort_values(by=['Name_x', 'Score'], ascending=False).groupby('Name_x').cumcount()

joined = joined.loc[(joined.RankY < 3) & (joined.RankX < 3)]
joined = joined.sort_values(by=['Handle','Name_x', 'Score'], ascending=[True,True,False])\
    .drop_duplicates(subset=['Handle'], keep='first')

# count the dups and join back
dups = joined[['Name_x', 'Body']].groupby('Name_x').count().reset_index().rename(columns={'Body':'ndups'})
dups = dups.loc[dups.ndups > 1]

joined = joined.merge(dups, on='Name_x', how='left').fillna(0)
joined.ndups = joined.ndups.astype(int)

print(len(joined))
joined.head(25)

370


,Position,StateName,Name_x,Name_y,Handle,Affiliation,Body,Score,RankY,RankX,ndups
0,GA-08,South Carolina,Austin Scott,Tim Scott,@AustinScottGA08,R,Senate,67,1,0,0
1,MS-02,Mississippi,Bennie Thompson,Bennie Thompson,@BennieGThompson,D,House of Representatives,100,0,0,0
2,MN-04,Minnesota,Betty McCollum,Betty McCollum,@BettyMcCollum04,D,House of Representatives,100,0,0,0
3,NJ-09,Texas,Bill Jr.,Will Hurd,@BillPascrell,R,House of Representatives,59,1,1,0
4,VA-03,Alabama,Robert Scott,Robert Aderholt,@BobbyScott,R,House of Representatives,67,0,0,0
5,State,California,Name Sherman,Brad Sherman,@BradSherman,D,House of Representatives,75,0,0,0
6,MD-02,Maryland,Dutch Ruppersberger,Dutch Ruppersberger,@Call_Me_Dutch,D,House of Representatives,100,0,0,0
7,TX-11,Texas,Michael Conaway,Mike Conaway,@ConawayTX11,R,House of Representatives,81,0,0,0
8,PA-02,Pennsylvania,Brendan Boyle,Brendan Boyle,@CongBoyle,D,House of Representatives,100,0,0,0
9,ID-02,Idaho,Michael Simpson,Mike Simpson,@CongMikeSimpson,R,House of Representatives,81,0,0,0


Write to a csv and do final cleaning by hand.

In [12]:
joined.to_csv('116thCongress_HandleAndParties.csv', index=None)